<font color="white">.</font> | <font color="white">.</font> | <font color="white">.</font>
-- | -- | --
![NASA](http://www.nasa.gov/sites/all/themes/custom/nasatwo/images/nasa-logo.svg) | <h1><font size="+3">ASTG Python Courses</font></h1> | ![NASA](https://www.nccs.nasa.gov/sites/default/files/NCCS_Logo_0.png)

---

<center>
    <h1><font color="red">Image Classification with Tensorflow</font></h1>
</center>

## Useful Reference

- [MNIST digits classification with TensorFlow 2](https://github.com/antonio-f/TensorFlow2_digits_classification-Linear_Classifier-MLP/blob/master/TensorFlow2_digits_classification-Linear_Classifier-MLP/digits_classification.ipynb)
- [Mnist handwritten digit classification using tensorflow](https://milindsoorya.site/blog/handwritten-digits-classification)
- [A real example – recognizing handwritten digits](https://subscription.packtpub.com/book/data/9781838823412/1/ch01lvl1sec08/a-real-example-recognizing-handwritten-digits)
- [DIFFERENCE BETWEEN SOFTMAX FUNCTION AND SIGMOID FUNCTION](https://dataaspirant.com/difference-between-softmax-function-and-sigmoid-function/)

### Load the modules

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [ ]:
print(f"Version of Numpy:      {np.__version__}")
print(f"Version of Pandas:     {pd.__version__}")
print(f"Version of Keras:      {tf.keras.__version__}")
print(f"Version of TensorFlow: {tf.__version__}")

# <font color="red">Image Classification</font> 

We use the [MNIST data set](http://yann.lecun.com/exdb/mnist/) (Modified National Institute of Standards and Technology database).

* Is a large database of handwritten digits that is commonly used for training various image processing systems.
* The database is also widely used for training and testing in the field of machine learning.
* The dataset we will be using contains 70000 images of handwritten digits among which 10000 are reserved for testing.
* It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting.

![TSF](https://static.javatpoint.com/tutorial/tensorflow/images/mnist-dataset-in-cnn.jpg)
Image Source: [https://www.javatpoint.com/tensorflow-mnist-dataset-in-cnn](https://www.javatpoint.com/tensorflow-mnist-dataset-in-cnn)

## <font color="blue"> Load MNiST Dataset</font>

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
print("Shape train inputs:  ", x_train.shape)
print("Shape train outputs: ", y_train.shape)
print("Shape test  inputs:  ", x_test.shape)
print("Shape test  outputs: ", y_test.shape)

In [ ]:
# Save image parameters to the constants that we will use later 
# for data re-shaping and for model traning.
(_, IMAGE_WIDTH, IMAGE_HEIGHT) = x_train.shape
print(f'IMAGE_WIDTH:  {IMAGE_WIDTH}')
print(f'IMAGE_HEIGHT: {IMAGE_HEIGHT}')

In [ ]:
print("Type train inputs:  ", x_train.dtype)
print("Type train outputs: ", y_train.dtype)
print("Type test  inputs:  ", x_test.dtype)
print("Type test  outputs: ", y_test.dtype)

In [ ]:
np.unique(x_train)

In [ ]:
np.unique(x_test)

In [ ]:
np.unique(y_train)

In [ ]:
np.unique(y_test)

In [ ]:
y_train.

#### Check an arbitrary image

In [ ]:
some_index = 15657
some_digit = x_train[some_index]
some_digit_image = some_digit.reshape(IMAGE_WIDTH, IMAGE_HEIGHT)

plt.imshow(some_digit_image, 
           cmap = matplotlib.cm.binary, 
           interpolation='nearest')
plt.axis=('off')

In [ ]:
print(y_train[some_index])

## <font color="blue"> Preprocess the Training and Test Datasets</font>

#### Change the data type

- Change the type from integer to floating point. 
- This will reduce our memory requirements by forcing the precision of the pixel values to be 32 bit, the default precision used by `Keras` anyway.

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [ ]:
print(f"Train --> min = {np.min(x_train)} max = {np.max(x_train)}")
print(f"Test  --> min = {np.min(x_test)} max = {np.max(x_test)}")

#### Normalize the data
- The values are from 0.0 to 255.0.
- We want to have values between 0.0 and 1.0.
- Normalizing the data generally speeds up trainning and leads to faster convergence.

In [ ]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
print(f"Train --> min = {np.min(x_train)} max = {np.max(x_train)}")
print(f"Test  --> min = {np.min(x_test)} max = {np.max(x_test)}")

#### Reshape the data

- The training and test datasets are structured as a 3-dimensional array of instance, image width and image height. 
- For a multi-layer perceptron model we must reduce the images down into a vector of pixels. In this case the `IMAGE_WIDTH*IMAGE_HEIGHT` sized images will be 784 pixel input values.
- We can do this transform easily using the `reshape()` function on the NumPy array.

In [ ]:
x_train_reshape = x_train.reshape(x_train.shape[0], 
                                  IMAGE_WIDTH*IMAGE_HEIGHT)
x_test_reshape = x_test.reshape(x_test.shape[0], 
                                IMAGE_WIDTH*IMAGE_HEIGHT)

#### Convert class vectors to binary class matrices

- The targets have 10 possible integer values: `0, 1, 2, ..., 9`.
- We use the `to_categorical` function to convert integer targets into categorical.
  - For instance, `2` would become `[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]` (it’s zero-indexed).
- We do it because `Keras` will expect the training targets to be 10-dimensional vectors, since there will be 10 nodes in the output layer.

In [ ]:
num_classes = 10
y_train_convert = tf.keras.utils.to_categorical(y_train, num_classes)
y_test_convert = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
print(y_train_convert[some_index])

## <font color="blue"> Model 1: Simple Sequential Model</font>

Architecture of the Network is:

1. Input layer for `IMAGE_WIDTH*IMAGE_HEIGHT = 28x28 = 784` images in MNiST dataset
2. Dense layer with 128 neurons and ReLU activation function
3. Output layer with 10 neurons for classification of input images as one of ten digits(0 to 9)

#### Remarks
- We add a regularization `Dropout` layer to randomly exclude a portion of the neurons (here 20%) in the layer in order to reduce overfitting.
- A `softmax` activation function is used on the output layer to turn the outputs into probability-like values and allow one class of the 10 to be selected as the model’s output prediction.
  - Converts the result into a probability distribution.
  - Calculates probabilities of each target class over all possible target classes
  - The values of the output vector are in range (0, 1) and sum to 1. 
  - `softmax` of input `x` is calculated by function `exp(x)/tf.reduce_sum(exp(x))`.

In [ ]:
one_layer_model = tf.keras.models.Sequential()
one_layer_model.add(tf.keras.layers.Dense(
    128, 
    activation='relu', 
    input_shape=(IMAGE_WIDTH*IMAGE_HEIGHT,)))
one_layer_model.add(tf.keras.layers.Dropout(0.2))
one_layer_model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [ ]:
one_layer_model.summary()

In [ ]:
#tf.keras.utils.plot_model(
#    one_layer_model,
#    show_shapes=True,
#    show_layer_names=True,
#)

### <font color="red">Compile the Model</font>

Before the model is ready for training, it needs a few more settings. These are added during the model's compile step:

- Loss function This measures how accurate the model is during training. You want to minimize this function to "steer" the model in the right direction.
- Optimizer This is how the model is updated based on the data it sees and its loss function.
- Metrics Used to monitor the training and testing steps. The following example uses accuracy, the fraction of the images that are correctly classified.

![MLP](https://m0nads.files.wordpress.com/2021/01/linear_classifier.png)
Image Source: m0nads.wordpress.com

In [ ]:
one_layer_model.compile(loss='categorical_crossentropy',
                        optimizer=tf.keras.optimizers.RMSprop(),
                        metrics=['accuracy'])

### <font color="red"> Training and Validation</font>

The `one_layer_model.fit` method adjusts the model parameters to minimize the loss:

In [ ]:
num_epochs = 10
batch_size = 16

In [ ]:
one_layer_history = one_layer_model.fit(
    x_train_reshape, 
    y_train_convert,
    batch_size = batch_size,
    epochs = num_epochs,
    verbose = 1, 
    validation_data = (x_test_reshape, y_test_convert))

### <font color="red"> Plot the Deceasing Loss over Epochs</font>

Use Pandas to plot a graph showing the decrease in mean squared error (mse) as training improves the model.

In [ ]:
hist_df = pd.DataFrame(one_layer_history.history)
hist_df

In [ ]:
hist_df[['loss', 'val_loss']].plot(xlabel='Epochs', 
                                   ylabel='Loss', 
                                   title='Loss');

In [ ]:
hist_df[['accuracy', 'val_accuracy']].plot(xlabel='Epochs', 
                                   ylabel='Accuracy', 
                                   title='Accuracy');

### <font color="red"> Evaluate the Model</font>

The `mnist_model.evaluate` method checks the models performance, usually on a "Validation-set" or "Test-set".

In [ ]:
score = one_layer_model.evaluate(x_test_reshape,  
                                 y_test_convert, 
                                 verbose=0)
print(f'Test loss:     {score[0]}')
print(f'Test accuracy: {score[1]}')

### <font color="red"> Visualize Predictions</font>

In [ ]:
def display_digits(X, y):
    """
      Given an array of images of digits X and 
      the corresponding values of the digit y,
      this function plots the first 96 images and their values.
    """
    # Figure size (width, height) in inches
    fig = plt.figure(figsize=(8, 6))

    # Adjust the subplots 
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

    for i in range(96):
        # Initialize the subplots: 
        #    Add a subplot in the grid of 8 by 12, at the i+1-th position
        ax = fig.add_subplot(8, 12, i + 1, xticks=[], yticks=[])
        
        # Display an image at the i-th position
        ax.imshow(X[i].reshape(28, 28), cmap=plt.cm.binary, interpolation='nearest')
       
        # label the image with the target value
        ax.text(0, 7, str(y[i]))

    # Show the plot
    plt.show()

In [ ]:
probabilities = one_layer_model.predict(x_test_reshape, steps=1)

In [ ]:
probabilities.shape

In [ ]:
probabilities[0]

We use the `numpy.argmax` function to return the indices of the maximum values along an axis.

In [ ]:
one_layer_predicted_labels = np.argmax(probabilities, axis=1)

In [ ]:
one_layer_predicted_labels[0]

In [ ]:
display_digits(x_test_reshape, one_layer_predicted_labels)

### <font color="red">Confusion Matrix for Validation</font>

- We can use the confusion matrix to have a picture of our prediction.
- A number `n` in a cell means that we predicted the value in the truth row as the value in the predicted column, `n` times. 
- All the diagonal elements are correct predictions.
- In the example below, the black cells, value shows the wrong predictions. 

In [ ]:
cm = tf.math.confusion_matrix(labels=y_test, 
                              predictions=one_layer_predicted_labels)

plt.figure(figsize = (10,7))
sns.heatmap(cm, annot=True, fmt='d');
plt.xlabel('Predicted');
plt.ylabel('Truth');

In [ ]:
def determine_accuracy(cmatrix):
    cm_values = cmatrix.numpy()
    l = cm_values.shape[0]
    diag = [cm_values[i, i] for i in range(l)]
    
    pds = pd.Series(y_test)

    org_total = pds.value_counts().sort_index()
    org_percent = (pds.value_counts()/pds.count()).sort_values().sort_index()*100
    pred_accu = (pd.Series(diag)/org_total)*100
    idx = ['zero', 'one', 'two', 'three', 'four',
          'five', 'six', 'seven', 'eight', 'nine']

    keys = ['Original Total', 'Predicted Total', 'Predicted Accuracy']
    accu_data = pd.concat([org_total, pd.Series(diag), pred_accu], 
                             axis=1, keys=keys)

    accu_data.index = idx
    return accu_data

In [ ]:
accuracy_data = determine_accuracy(cm)
accuracy_data

### <font color="red">Save the Model</font>

In [ ]:
one_layer_model.save('one_layer_model')

Then to reload the model later, we can use this:

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('one_layer_model')

## <font color="blue"> Model 2: Add a Hidden Layer</font>

In [ ]:
two_layer_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(IMAGE_WIDTH*IMAGE_HEIGHT, )),
    tf.keras.layers.Dense(256, activation=tf.nn.sigmoid),
    tf.keras.layers.Dense(256, activation=tf.nn.sigmoid),
    tf.keras.layers.Dense(10)])

In [ ]:
two_layer_model.summary()

In [ ]:
two_layer_model.compile(optimizer='adam',
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           metrics=['accuracy'])

In [ ]:
two_layer_history = two_layer_model.fit(
    x_train_reshape, 
    y_train, 
    batch_size = batch_size,
    epochs=num_epochs)

In [ ]:
hist_df = pd.DataFrame(two_layer_history.history)
hist_df

In [ ]:
hist_df['loss'].plot(xlabel='Epochs', 
                     ylabel='Loss', 
                     title='Loss');

In [ ]:
hist_df['accuracy'].plot(xlabel='Epochs', 
                     ylabel='Accuracy', 
                     title='Accuracy');

In [ ]:
score = two_layer_model.evaluate(x_test_reshape,  
                                 y_test, 
                                 verbose=0)
print(f'Test loss:     {score[0]}')
print(f'Test accuracy: {score[1]}')

In [ ]:
predictions = two_layer_model.predict(x_test_reshape)
two_layer_predicted_labels = np.argmax(predictions, axis=1)

In [ ]:
print(y_test[:10])
print(two_layer_predicted_labels[:10])

In [ ]:
cm = tf.math.confusion_matrix(labels=y_test, 
                              predictions=two_layer_predicted_labels)

plt.figure(figsize = (10,7))
sns.heatmap(cm, annot=True, fmt='d');
plt.xlabel('Predicted');
plt.ylabel('Truth');

In [ ]:
accuracy_data = determine_accuracy(cm)
accuracy_data